In [1]:
import numpy as np
from numpy import random as rnd
from matplotlib import pyplot as plt 
import os,warnings,datetime,time,math,itertools

import pandas as pd

import torch
import torch.nn as nn 
import torch.nn.functional as F 

import plotly.express as px
import graphdb as gdb

from SPARQLWrapper import SPARQLWrapper,JSON

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Connect to Wikidata

A first attempt to connect to Wikidata and fire off a simple query

In [2]:
sparql = SPARQLWrapper('https://query.wikidata.org/sparql')

In [3]:
sparql.setQuery(
'''
SELECT ?item ?itemLabel 
WHERE 
{
  ?item wdt:P31 wd:Q146 .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". } 
}
'''
)
sparql.setReturnFormat(JSON)


In [4]:
results = sparql.query().convert()

In [5]:
results_df = pd.io.json.json_normalize(results['results']['bindings'])

/var/folders/3l/h5txsrx128z_kjh049xwhtqh0000gn/T/ipykernel_1584/1614311846.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  results_df = pd.io.json.json_normalize(results['results']['bindings'])


In [6]:
results_df.shape

(160, 5)

In [7]:
results_df.head()

,item.type,item.value,itemLabel.xml:lang,itemLabel.type,itemLabel.value
0,uri,http://www.wikidata.org/entity/Q378619,en,literal,CC
1,uri,http://www.wikidata.org/entity/Q498787,en,literal,Muezza
2,uri,http://www.wikidata.org/entity/Q677525,en,literal,Orangey
3,uri,http://www.wikidata.org/entity/Q851190,en,literal,Mrs. Chippy
4,uri,http://www.wikidata.org/entity/Q893453,en,literal,Unsinkable Sam


# Connect to DBPedia

In [16]:
sparql = SPARQLWrapper('https://dbpedia.org/sparql')
sparql.setQuery('''
PREFIX dbo: <http://dbpedia.org/ontology/> 
PREFIX foaf: <http://xmlns.com/foaf/0.1/> 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 

SELECT ?country ?city ?city_name
WHERE {
    ?city rdf:type dbo:City ;
          foaf:name ?city_name ;
          dbo:country ?country .
    FILTER(langMatches(lang(?city_name), "en"))
}
ORDER BY ?city_name
'''
)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

In [17]:
dbpdia_result = pd.io.json.json_normalize(results['results']['bindings'])
dbpdia_result

/var/folders/3l/h5txsrx128z_kjh049xwhtqh0000gn/T/ipykernel_1584/673463325.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  dbpdia_result = pd.io.json.json_normalize(results['results']['bindings'])


,country.type,country.value,city.type,city.value,city_name.type,city_name.xml:lang,city_name.value
0,uri,http://dbpedia.org/resource/Canada,uri,http://dbpedia.org/resource/Amqui,literal,en,
1,uri,http://dbpedia.org/resource/Ukraine,uri,http://dbpedia.org/resource/Ustyluh,literal,en,
2,uri,http://dbpedia.org/resource/Ukraine,uri,http://dbpedia.org/resource/Vashkivtsi,literal,en,
3,uri,http://dbpedia.org/resource/Ukraine,uri,http://dbpedia.org/resource/Velyki_Mosty,literal,en,
4,uri,http://dbpedia.org/resource/Greece,uri,http://dbpedia.org/resource/Veria,literal,en,
...,...,...,...,...,...,...,...
9995,uri,http://dbpedia.org/resource/Canada,uri,"http://dbpedia.org/resource/Léry,_Quebec",literal,en,Léry
9996,uri,http://dbpedia.org/resource/Canada,uri,"http://dbpedia.org/resource/Lévis,_Quebec",literal,en,Lévis
9997,uri,http://dbpedia.org/resource/http://dbpedia.org...,uri,http://dbpedia.org/resource/Löbau,literal,en,Löbau
9998,uri,http://dbpedia.org/resource/http://dbpedia.org...,uri,http://dbpedia.org/resource/Lörrach,literal,en,Lörrach


# Connect to local GraphDB instance

Instance running at 'http://localhost:7200/sparql'

In [18]:
sparql = SPARQLWrapper('http://Jan-Lucass-MBP:7200/repositories/DQTest')

In [29]:
sparql.setQuery(
'''
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX gn: <http://www.geonames.org/ontology#>
PREFIX geo: <http://telegraphis.net/ontology/geography/geography#>
PREFIX ctry: <http://telegraphis.net/data/countries/>

SELECT * 
WHERE { 
	?s geo:capital ?o .
	FILTER(?s IN (<http://telegraphis.net/data/countries/FR#FR>,<http://telegraphis.net/data/countries/DE#DE>)) .
}
'''
)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

results_df = pd.io.json.json_normalize(results['results']['bindings'])
results_df

/var/folders/3l/h5txsrx128z_kjh049xwhtqh0000gn/T/ipykernel_1584/2345879232.py:19: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  results_df = pd.io.json.json_normalize(results['results']['bindings'])


,s.type,s.value,o.type,o.value
0,uri,http://telegraphis.net/data/countries/FR#FR,uri,http://telegraphis.net/data/capitals/FR/Paris#...
1,uri,http://telegraphis.net/data/countries/DE#DE,uri,http://telegraphis.net/data/capitals/DE/Berlin...
